In [2]:
import sparknlp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from pyspark.sql import SparkSession
from pyspark.sql import Row
# from pyspark.sql.types import StringType, StructType, StructField
from pyspark.sql.types import *
# import pyspark.sql.functions as F
from pyspark.sql.functions import *

import warnings
warnings.filterwarnings('ignore')

spark = sparknlp.start()

:: loading settings :: url = jar:file:/opt/module/spark-3.5.0-bin-hadoop3/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
com.johnsnowlabs.nlp#spark-nlp_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-a81ab276-4a86-4413-9d78-81c3f91f70d2;1.0
	confs: [default]
	found com.johnsnowlabs.nlp#spark-nlp_2.12;3.3.0 in central
	found com.typesafe#config;1.4.1 in central
	found org.rocksdb#rocksdbjni;6.5.3 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.603 in central
	found com.github.universal-automata#liblevenshtein;3.0.0 in central
	found com.google.code.findbugs#annotations;3.0.1 in central
	found net.jcip#jcip-annotations;1.0 in central
	found com.google.code.findbugs#jsr305;3.0.1 in central
	found com.google.protobuf#protobuf-java-util;3.0.0-beta-3 in central
	found com.google.protobuf#protobuf-java;3.0.0-beta-3 in central
	found com.google.code.gson#gson;2.3 in central
	found it.unimi.dsi#fastutil;7.0.12 in central
	found org.projectlombok#lombok;1.16.8 in c

In [3]:
spark = SparkSession.builder \
    .appName("Spark NLP") \
    .master("local[*]") \
    .config("spark.driver.memory", "16G") \
    .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer") \
    .config("spark.kryoserializer.buffer.max", "2000M") \
    .config("spark.driver.maxResultSize", "0") \
    .config("spark.jars.packages", "com.johnsnowlabs.nlp:spark-nlp_2.12:5.3.3") \
    .getOrCreate()

24/06/03 19:56:39 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [4]:
import transformers

model_name_path = "/STA323_zc/proj2/flan-t5-small"
EXPORT_PATH = f"nlp_{model_name_path.split('/')[-1]}"

In [5]:
!optimum-cli export onnx --task text2text-generation-with-past --model {model_name_path} {EXPORT_PATH}

/root/anaconda3/envs/sp/lib/python3.11/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/root/anaconda3/envs/sp/lib/python3.11/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
Framework not specified. Using pt to export the model.
Using the export variant default. Available variants are:
    - default: The default ONNX variant.

***** Exporting submodel 1/3: T5Stack *****
Using framework PyTorch: 2.3.0+cpu
Overriding 1 configuration item(s)
	- use_cache -> False

***** Exporting submodel 2/3: T5ForConditionalGeneration *****
Using framework PyTorch: 2.3.0+cpu
Overriding 1 configuration item(s)
	- use_cache -> True
/root/anaconda3/envs/sp/lib/python3.11

In [12]:
!ls -l {EXPORT_PATH}

! mkdir -p {EXPORT_PATH}/assets
! mv -t {EXPORT_PATH}/assets {EXPORT_PATH}/assets/spiece.model

total 809792
drwxr-xr-x 2 root root      4096 Jun  3 19:59 assets
-rw-r--r-- 1 root root      1537 Jun  3 19:56 config.json
-rw-r--r-- 1 root root 232553676 Jun  3 19:58 decoder_model.onnx
-rw-r--r-- 1 root root 232784389 Jun  3 19:58 decoder_model_merged.onnx
-rw-r--r-- 1 root root 219953983 Jun  3 19:58 decoder_with_past_model.onnx
-rw-r--r-- 1 root root 141456358 Jun  3 19:57 encoder_model.onnx
-rw-r--r-- 1 root root       142 Jun  3 19:56 generation_config.json
-rw-r--r-- 1 root root      2201 Jun  3 19:56 special_tokens_map.json
-rw-r--r-- 1 root root   2422256 Jun  3 19:56 tokenizer.json
-rw-r--r-- 1 root root     20771 Jun  3 19:56 tokenizer_config.json


mv: 'nlp_flan-t5-small/assets/spiece.model' and 'nlp_flan-t5-small/assets/spiece.model' are the same file


In [13]:
!ls -l {EXPORT_PATH}/assets

total 776
-rw-r--r-- 1 root root 791656 Jun  3 19:56 spiece.model


In [8]:
! wget -q http://setup.johnsnowlabs.com/colab.sh -O - | bash

Installing PySpark 3.2.3 and Spark NLP 5.3.3
setup Colab for PySpark 3.2.3 and Spark NLP 5.3.3


In [14]:
import sparknlp

# let's start Spark with Spark NLP
spark = sparknlp.start()

In [15]:
from sparknlp.annotator import *

T5 = T5Transformer.loadSavedModel(EXPORT_PATH, spark)\
  .setUseCache(True) \
  .setTask("summarize:") \
  .setMaxOutputLength(200)

IllegalArgumentException: requirement failed: savedModel file saved_model.pb not found in folder nlp_flan-t5-small

In [ ]:
T5.write().overwrite().save(f"{MODEL_NAME}_spark_nlp")

In [ ]:
!rm -rf {EXPORT_PATH}

In [ ]:
import sparknlp
from sparknlp.base import *
from sparknlp.annotator import *
from pyspark.ml import Pipeline

test_data = spark.createDataFrame([
    ["Transfer learning, where a model is first pre-trained on a data-rich task before being fine-tuned on a " +
       "downstream task, has emerged as a powerful technique in natural language processing (NLP). The effectiveness" +
       " of transfer learning has given rise to a diversity of approaches, methodology, and practice. In this " +
       "paper, we explore the landscape of transfer learning techniques for NLP by introducing a unified framework " +
       "that converts all text-based language problems into a text-to-text format. Our systematic study compares " +
       "pre-training objectives, architectures, unlabeled data sets, transfer approaches, and other factors on dozens " +
       "of language understanding tasks. By combining the insights from our exploration with scale and our new " +
       "Colossal Clean Crawled Corpus, we achieve state-of-the-art results on many benchmarks covering " +
       "summarization, question answering, text classification, and more. To facilitate future work on transfer " +
       "learning for NLP, we release our data set, pre-trained models, and code."]
]).toDF("text")


document_assembler = DocumentAssembler() \
    .setInputCol("text")\
    .setOutputCol("document")

T5 = T5Transformer.load(f"{MODEL_NAME}_spark_nlp") \
  .setInputCols(["document"]) \
  .setOutputCol("summary")

pipeline = Pipeline().setStages([document_assembler, T5])

result = pipeline.fit(test_data).transform(test_data)
result.select("summary.result").show(truncate=False)